In [1]:
from ib_insync import *
util.startLoop()

ib = IB()
ib.connect('127.0.0.1', 7497, clientId=15)

<IB connected to 127.0.0.1:7497 clientId=15>

In [2]:
# from ib_insync import *
import numpy as np
import pandas as pd
from scipy import stats
import matplotlib.pyplot as plt
import math
import datetime 
import statsmodels.formula.api as smf 
from datetime import date, time, datetime, timedelta
import seaborn as sns
import random
from ta import add_all_ta_features
from ta.utils import dropna
from ta.trend import *
from collections import deque, defaultdict

In [5]:
##### Initialization of global variables here ###

order_books = defaultdict(set)
df_ratio = pd.read_csv("ratio_dict.csv", index_col = 0).set_index("Pair")
df_global = pd.DataFrame({"date":[]})


country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "China":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

In [6]:
def place_order(country, security, sectype, side, quantity, price):
    global order_books
    exchange = country_info[country]["Exchange"]
    currenct = country_info[country]["Currency"]
    
    # Define order 
    contract = Contract(symbol = security, secType = 'STK', exchange =exchange, currency = currency)
    order = LimitOrder(side,quantity,price)
    
    # Place the order
    msg = ib.placeOrder(contract, order)
    orderId = msg.order.orderId
    
    # Add the orderId to orderbook
    name =  security +"_"+ sectype
    order_books[name].add(orderId)
    
    

def check_open_order(security, sectype = "underlying"):
    """
    Check if we have open order for this particular security
    Args: 
    security (str) name of security
    sectype(str): takes th evalue either "adr" or "underlying"
    
    return: the quantity of open orders (orders that we wanted to trade but not traded) of a particular security
    
    """
    global order_books
    ct = 0
    lookup = security +"_"+ sectype
    all_orders = order_books[lookup]
    open_orders = ib.openOrders()
    for o in openOders:
        if o.orderId in all_orders:
            ct += o.totalQuantity
        
    return ct
        

In [35]:
df_ratio

,adr_num_per_stock,stock_num_per_adr
Pair,,
ATHE_ATH,0.016667,60.0
GENE_GTG,0.001667,600.0
IMMP_IMM,0.100000,10.0
IMRN_IMC,0.025000,40.0
JHX_JHX,1.000000,1.0
KZIA_KZA,0.100000,10.0
MESO_MSB,0.200000,5.0
PLL_PLL,0.010000,100.0
WBK_WBC,1.000000,1.0


In [34]:
df_global

,date,underlying_1055_close,underlying_386_close,underlying_338_close,underlying_857_close,underlying_2628_close,underlying_902_close,underlying_670_close,underlying_6160_close,underlying_2600_close,...,adr_CAJ_close,adr_WBK_close,adr_PLL_close,adr_MESO_close,adr_KZIA_close,adr_JHX_close,adr_IMRN_close,adr_IMMP_close,adr_GENE_close,adr_ATHE_close
9,2021-04-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.28,19.22,72.67,8.56,14.71,32.23,6.00,3.54,4.29,1.41
10,2021-04-09,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,23.54,19.25,69.30,8.51,13.62,32.84,5.89,3.53,4.57,1.39
0,2021-04-12,5.55,4.07,1.94,2.72,15.80,2.72,3.64,183.3,3.48,...,23.78,19.36,67.03,8.45,12.13,31.97,5.63,3.50,4.21,1.28
1,2021-04-13,5.50,4.17,1.94,2.74,15.76,2.67,3.60,178.0,3.59,...,23.49,19.33,69.51,8.64,12.13,32.03,5.68,3.57,4.23,1.26
2,2021-04-14,5.52,4.26,1.96,2.78,15.78,2.69,3.58,179.5,3.74,...,24.03,19.43,68.00,8.70,11.42,32.55,5.53,3.48,4.09,1.32
3,2021-04-15,5.51,4.20,1.96,2.81,15.54,2.70,3.56,180.0,3.92,...,24.12,19.71,67.24,9.00,11.33,33.73,5.59,3.37,3.90,1.27
4,2021-04-16,5.59,4.27,1.97,2.83,15.74,2.73,3.64,185.3,3.79,...,23.92,19.68,65.07,8.94,11.33,33.42,5.37,3.29,3.87,1.28
5,2021-04-19,5.77,4.27,1.99,2.84,15.74,2.77,3.64,185.5,3.87,...,23.91,19.55,65.98,8.81,10.98,33.59,5.41,3.29,3.85,1.28
6,2021-04-20,5.70,4.25,1.99,2.81,15.74,2.76,3.63,182.8,3.90,...,23.50,19.33,63.68,8.55,10.79,32.93,5.35,3.18,3.79,1.25
7,2021-04-21,5.52,4.00,1.95,2.74,15.58,2.74,3.56,177.9,3.85,...,23.44,19.51,67.57,8.85,10.57,33.86,5.40,3.21,3.83,1.44


In [22]:
df_forex

,date,JPY.USD,AUD.USD,HKD.USD
0,2020-11-30 17:15:00,0.009587,0.734580,0.129011
1,2020-11-30 17:16:00,0.009587,0.734490,0.129011
2,2020-11-30 17:17:00,0.009587,0.734445,0.129011
3,2020-11-30 17:18:00,0.009587,0.734460,0.129011
4,2020-11-30 17:19:00,0.009586,0.734565,0.129011
...,...,...,...,...
979016,2021-04-22 05:09:00,0.009256,0.774430,0.128873
979017,2021-04-22 05:10:00,0.009255,0.774365,0.128873
979018,2021-04-22 05:11:00,0.009256,0.774460,0.128873
979019,2021-04-22 05:12:00,0.009256,0.774495,0.128873


In [33]:
df_ratio

,adr_num_per_stock,stock_num_per_adr
Pair,,
ATHE_ATH,0.016667,60.0
GENE_GTG,0.001667,600.0
IMMP_IMM,0.100000,10.0
IMRN_IMC,0.025000,40.0
JHX_JHX,1.000000,1.0
KZIA_KZA,0.100000,10.0
MESO_MSB,0.200000,5.0
PLL_PLL,0.010000,100.0
WBK_WBC,1.000000,1.0


In [16]:
def convert_price_asian_close():
    
    """
    Convert the last Asian prices we pulled before US open into US dollars
    Using the most recent forex data
    
    """
    global df_global, df_forex, df_ratio, country_info
    
    # We use the last 
    for key in country_info:
        pairs = country_info[key]["Pairs"]
        for pair in pairs:
            print(pair)
            underlying_col_name = f"underlying_{pair[1]}_close"
            pair_name =  pair[0] + "_" + pair[1]
            print(pair_name)
            ratio = df_ratio.loc[pair_name,"adr_num_per_stock"] # Determing number of adr per stock
            
            forex_col_name = country_info[key]["Currency"]+".USD"
            fx = df_forex.loc[len(df_forex)-1, forex_col_name]
            price_in_usd = df_global.loc[len(df_global)-1, underlying_col_name]*ratio*fx
            
            
            df_global.at[len(df_global)-1, underlying_col_name] = price_in_usd

In [8]:
#define a function to get IB data. endDate is the last date of the historical data 

def get_data(contract, history, freq, side, endDate =''):
    bar = ib.reqHistoricalData(
        contract,
        endDateTime = endDate,
        durationStr=history,
        barSizeSetting= freq,
        whatToShow=side,
        useRTH=True,
        formatDate=1)
    return util.df(bar)


In [4]:
# Initialization of hard coded data

aus_adr_underlying_pairs = [
    ["ATHE", "ATH"],["GENE", "GTG"],["IMMP", "IMM"],["IMRN","IMC"],["JHX", "JHX"],["KZIA", "KZA"],
    ["MESO","MSB"],["PLL", "PLL"],["WBK","WBC"]
]

hk_adr_underlying_pairs = [
    ["ACH", "2600"],["BGNE", "6160"],["CEA","670"],["HNP", "902"],["LFC", "2628"],["PTR", "857"],
    ["SHI", "338"],["SNP","386"],["ZNH", "1055"]
]

jpn_adr_underlying_pairs = [
    ["CAJ", "7751"],["HMC","7267"],["IX","8591"],["MFG","8411"],["MUFG","8306"],["NMR","8604"],
    ["SMFG","8316"],["SONY","6758"],["TAK", "4502"],["TM", "7203"]
]

country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}

forex_pairs = [("AUD", "USD"),  ("USD", "HKD"), ("USD","JPY")]


# functions return the market opening time adjusted for DST

us_dst = {
        2021: (datetime(2021, 3, 14), datetime(2021, 11, 7)),
        2020: (datetime(2020, 3, 8),  datetime(2020, 11, 1)),
        2019: (datetime(2019, 3, 10), datetime(2019, 11, 3)),
        2018: (datetime(2018, 3, 11), datetime(2018, 11, 4)),
        2017: (datetime(2017, 3, 12), datetime(2017, 11, 5)),
        2016: (datetime(2016, 3, 13), datetime(2016, 11, 6)),
        2015: (datetime(2015, 3, 9),  datetime(2015, 11, 2)),
    } 

def next_market_open_day(date_now):
    '''
    Returns today if Mon - Thurs, returns next Mon if Fri thru Sun
    '''
    while date_now.weekday()>4:
        date_now += timedelta(days = 1)
    return date_now

def AUS_opening(date_now):
    date_now = next_market_open_day(date_now)
    us_start, us_end = us_dst[date_now.year]
    if us_start < date_now and date_now < us_end:
        return datetime.combine(date_now,time(10,0)) - timedelta(hours = 14)
    else:
        return datetime.combine(date_now, time(10,0)) - timedelta(hours = 15)

def AUS_closing(date_now):
    open_time = AUS_opening(date_now)
    return open_time + timedelta(hours = 6)

def HK_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,30)) - timedelta(hours = 12)
    else:
        return datetime.combine(date_now, time(9,30)) - timedelta(hours = 13)
    
def HK_closing(date_now):
    open_time = HK_opening(date_now)
    return open_time + timedelta(hours = 6.5)
    
def JPN_opening(date_now):
    date_now = next_market_open_day(date_now)
    start, end = us_dst[date_now.year]
    if start < date_now and date_now < end:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 13)
    else:
        return datetime.combine(date_now,time(9,0)) - timedelta(hours = 14)
    
def JPN_closing(date_now):
    open_time = JPN_opening(date_now)
    return open_time + timedelta(hours = 6)

def US_opening(date_now):
    return datetime.combine(next_market_open_day(date_now),time(9,30))

def US_closing(date_now):
    return datetime.combine(next_market_open_day(date_now),time(16,0))

In [36]:
df_ratio

,adr_num_per_stock,stock_num_per_adr
Pair,,
ATHE_ATH,0.016667,60.0
GENE_GTG,0.001667,600.0
IMMP_IMM,0.100000,10.0
IMRN_IMC,0.025000,40.0
JHX_JHX,1.000000,1.0
KZIA_KZA,0.100000,10.0
MESO_MSB,0.200000,5.0
PLL_PLL,0.010000,100.0
WBK_WBC,1.000000,1.0


Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has been lost.
Error 1100, reqId -1: Connectivity between IB and Trader Workstation has bee

In [29]:
AUS_opening(datetime(2021, 4, 16, 5))

datetime.datetime(2021, 4, 15, 20, 0)

In [9]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


# Store all the historical adr/underlying open and close data


def pull_asian_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_asian_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = country_info[key]["Exchange"]
        currency = country_info[key]["Currency"]
        
        for p in pairs:
            underlying = p[1]
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            try:
                df = get_data(contract, history, freq, side, endDate =today)
                df = df.loc[:, ["date", "close"]]
                col_name = "underlying_" + underlying + "_close"
                df = df.rename(columns = {"close":col_name})
#                 print(df)
                df_asian_close = pd.merge(df, df_asian_close, how = "outer")
            except:
                pass
            
    df_global = pd.concat([df_global, df_asian_close]).sort_values("date")

    
    
def pull_us_close(days = 1):
    global df_global, country_info

    today = datetime.now().date()
    history = f"{days} D"
    freq = "1 Day"
    side = 'Trades'
    
    df_us_close = pd.DataFrame({"date":[]})
    for key in country_info.keys():
        pairs = country_info[key]["Pairs"]
        exchange = "SMART"
        currency = "USD"
        
        for p in pairs:
            underlying = p[0] # get the ticker of the ADR
            contract = Contract(symbol = underlying, secType = 'STK', exchange = exchange, currency = currency)
            df = get_data(contract, history, freq, side, endDate ='')
            df = df.loc[:, ["date", "close"]]
            col_name = "adr_" + underlying + "_close"
            df = df.rename(columns = {"close":col_name})
            df_us_close = pd.merge(df, df_us_close, how = 'outer')
            
    df_global = pd.merge(df_global, df_us_close,how = 'outer').sort_values("date")


def pull_forex_data(history = "1 D", freq = "1 min", side = "BID_ASK", end_date = "" ):
    '''
    Updates all three forex pairs
    '''
    
    global df_global, country_info, forex_pairs, test, df_forex

    df_temp = pd.DataFrame({"date":[]})
    for currency1, currency2 in forex_pairs:
        contract = Contract(symbol = currency1, secType = 'CASH', exchange = "IDEALPRO", currency = currency2)
        df = get_data(contract, history, freq, side)

        if currency2 == "USD":
            df[f"{currency1}.{currency2}"] = (df["open"] + df["close"])/2
            forex_string = f"{currency1}.{currency2}"
        else:
            df[f"{currency2}.{currency1}"] = 2/(df["open"] + df["close"])
            forex_string = f"{currency2}.{currency1}"
        df_temp = pd.merge(df_temp, df[['date',forex_string]], how = "outer")
    
    df_forex = pd.merge(df_forex, df_temp, how = "outer")

In [11]:
pull_asian_close(10)
pull_us_close(10)
pull_forex_data()
convert_price_asian_close()

KeyError: 'C_u'

In [196]:

# test = df_global[["date", "adr_ACH_close", "underlying_2600_close"]]
# df_global

In [197]:
test

,date,adr_ACH_close,underlying_2600_close
9,2021-04-08,12.21,NaN
10,2021-04-09,11.83,NaN
0,2021-04-12,11.08,3.48
1,2021-04-13,11.55,3.59
2,2021-04-14,11.90,3.74
3,2021-04-15,12.47,3.92
4,2021-04-16,12.20,3.79
5,2021-04-19,12.26,3.87
6,2021-04-20,12.18,3.90
7,2021-04-21,12.48,3.85


In [209]:
# Pairs we have chosen to trade
trading_limits = {"Australia" : 4, "China" : 1, "Japan" : 3}
allocation = {"Australia" : 0.3, "China" : 0.4, "Japan" : 0.3}

list_pairs = [("Australia", "PLL_PLL"),
               ("Australia", "MESO_MSB"),
               ("Australia", "GENE_GTG"),
               ("Australia", "WBK_WBC"),
               ("Australia", "KZIA_KZA"),
               ("Australia", "IMMP_IMM"),
               ("Australia", "IMRN_IMC"),
               ("Australia", "ATHE_ATH"),
               ("Australia", "JHX_JHX"),
               ("Japan", "SONY_6758"), 
               ("Japan", "TAK_4502"),
               ("Japan", "TM_7203"),
               ("Japan", "SMFG_8316"),
               ("China", "BGNE_6160"), 
               ("China", "SNP_386")]


fname = 'results1_sfx_all.txt'
with open(fname, 'r') as f:
    is_res = f.readlines()
hp = {}
for i in range(len(list_pairs)):
    hp[list_pairs[i]] = [float(x) for x in is_res[i*5 + 4].split("(")[1].split(")")[0].split(", ")]
hp_dict = {}
for (country, adr) in list_pairs:
    hp_dict[(country, adr)] = {}
    hp_dict[(country, adr)]["lookback"] = int(hp[(country, adr)][0])
    hp_dict[(country, adr)]["entry"] = hp[(country, adr)][1]
    hp_dict[(country, adr)]["exit"] = hp[(country, adr)][2]
    hp_dict[(country, adr)]["stop_loss"] = hp[(country, adr)][3]
    # Fraction of cash allocated to each adr-stock pair
    hp_dict[(country, adr)]["allocation"] = allocation[country]/trading_limits[country]
    hp_dict[(country, adr)]["original_allocation"] = allocation[country]/trading_limits[country]

diff_record_dict = {}
for (country, adr) in list_pairs:
    diff_record_dict[(country, adr)] = deque(maxlen = 2*hp_dict[(country, adr)]["lookback"])

insight_dict = {}
for (country, adr) in list_pairs:
    insight_dict[(country,adr)] = "FLAT"
    
positions = {}
for (country, adr) in list_pairs:
    positions[(country, adr)] = {}
    positions[(country, adr)]["stock_pos"] = 0
    positions[(country, adr)]["adr_pos"] = 0
    positions[(country, adr)]["prev_adr_pos"] = 0
    positions[(country, adr)]["holding_period"] = None
    positions[(country, adr)]["trade_type"] = None


{('Australia', 'PLL_PLL'): deque([], maxlen=200), ('Australia', 'MESO_MSB'): deque([], maxlen=200), ('Australia', 'GENE_GTG'): deque([], maxlen=200), ('Australia', 'WBK_WBC'): deque([], maxlen=120), ('Australia', 'KZIA_KZA'): deque([], maxlen=120), ('Australia', 'IMMP_IMM'): deque([], maxlen=200), ('Australia', 'IMRN_IMC'): deque([], maxlen=60), ('Australia', 'ATHE_ATH'): deque([], maxlen=120), ('Australia', 'JHX_JHX'): deque([], maxlen=200), ('Japan', 'SONY_6758'): deque([], maxlen=200), ('Japan', 'TAK_4502'): deque([], maxlen=60), ('Japan', 'TM_7203'): deque([], maxlen=200), ('Japan', 'SMFG_8316'): deque([], maxlen=200), ('China', 'BGNE_6160'): deque([], maxlen=120), ('China', 'SNP_386'): deque([], maxlen=200)}


In [211]:
def update_z_score():
    global df_global, insight_dict
    for country, adr in list_pairs:
        ticker_ADR, ticker_underlying = adr.split("-")
        lookback_window = diff_record_dict[(country,adr)]
        # initialize lookback window for each stock
        if len(lookback_window) != lookback_window.maxlen:
            adr_price = df_global[f"adr_{ticker_ADR}_close"]
            underlying_price = df_global[f"adr_{ticker_underlying}_close"]
            us_bef_diff = adr_price-underlying_price
            us_aft_diff = (adr_price - underlying_price.shift(-1)).dropna()
            i = 1
            while diff_record_dict[(country,adr)].maxlen != len(diff_record_dict[(country,adr)]):
                lookback_window.appendleft(us_bef_diff.iloc[-i])
                lookback_window.appendleft(us_aft_diff.iloc[-i])
                i += 1
    
        # calculate Z score
        mean = np.array(lookback_window).mean()
        std = np.array(lookback_window).std()
        new_diff = adr_price.iloc[-1] - underlying_price.iloc[-1]
        zscore = ( new_diff - mean)/std
        
        # update lookback window
        lookback_window.append(new_diff)
        
        # make trading decisions
        if (zscore > hp_dict[(country, adr)]["entry"]):
            insight_dict[(country, adr)] = "SHORT"
        elif (zscore < hp_dict[(country, adr)]["entry"]):
            insight_dict[(country,adr)] = "LONG"
        elif (zscore <  -hp_dict[(country, adr)]["exit"]):
            insight_dict[(country, adr)] = "LIQUID LONG"
        elif (zscore > -hp_dict[(country, adr)]["exit"]):
            insight_dict[(country, adr)] = "LIQUID SHORT"
        else: 
            insight_dict[(country, adr)] = "FLAT"
        

SyntaxError: invalid syntax (<ipython-input-211-f64a12f779e3>, line 28)

In [18]:
currency1 = "AUD"
secType = "CASH"
currency2 = "USD"
exchange = "IDEALPRO"

history = "1 D"
freq = "1 min"
side = "BID_ASK"

forex_contract = Contract(symbol = currency1, secType = secType, exchange = exchange, currency = currency2)
ib.qualifyContracts(forex_contract)
df_forex = get_data(forex_contract, history, freq, side)


In [ ]:
# 8.30 am
    # Pull Historical Forex Data (Minute, for lookback window)
    # Pull Close Data for Asian Market at 9am (Before US Market opens)
# 9.28 am 
    # Pull Forex Data for three currencies (For Z score)
    # Calculate Z Score 
    # for each pair:
        # if invested:
            # if risk condition or exit condition:
                # liquidate
            # else:
                # continue
        # if entry condition:
            # determine order size from risk metric and ADT over last 5 days
            # if 0: no order made
            # else: 
                # Define contract
                # make order (What order? Market Order?)


# 6 pm
    # Pull Close Data for US Market at 6pm (Before Asian Market opens)
    # Pull Historical Forex Data

# 2 min before Asian Market Open
    # Pull Forex Data
    # Calculate Z score using market close and minute forex data

In [10]:
# initialize data lists, hyperparameters etc

path_data = "live_trading_data/"
df_forex = pd.DataFrame({"date":[]})
def initialize_forex():
    global df_forex, country_info

    now = datetime.now()
    df_forex = pd.read_csv(f"{path_data}df_forex.csv")
    df_forex["date"] = [datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S') for date_time in df_forex["date"]]
    pull_head = max(now-timedelta(days = round(7/5*100)),df_forex["date"].iloc[-1])


    now = datetime.now()
    df_temp_2 = pd.DataFrame({"date":[]})
    while pull_head < now:
        print(pull_head.date())
        for currency1, currency2 in forex_pairs:

            history = "1 D"
            freq = "1 min"
            side = "BID_ASK"

            forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
            ib.qualifyContracts(forex_contract)
            df_temp = get_data(forex_contract, history, freq, side)

            if currency2 == "USD":
                df_temp[f"{currency1}.{currency2}"] = (df_temp["open"] + df_temp["close"])/2
                forex_string = f"{currency1}.{currency2}"
            else:
                df_temp[f"{currency2}.{currency1}"] = 2/(df_temp["open"] + df_temp["close"])
                forex_string = f"{currency2}.{currency1}"

            df_temp["date"] = df_temp["date"].astype('datetime64[s]')
            df_temp_2 = pd.merge(df_temp_2, df_temp[["date",forex_string]], how = "outer")



        pull_head += timedelta(days = 1)
    df_forex = pd.merge(df_forex,df_temp_2, how = "outer")
    df_forex.to_csv(f"{path_data}df_forex.csv", index = False)
    print("df_forex initialized")


,date,JPY.USD,AUD.USD,HKD.USD
978860,2021-04-22 02:33:00,0.009260,0.775145,0.128858
978861,2021-04-22 02:34:00,0.009261,0.775290,0.128858
978862,2021-04-22 02:35:00,0.009262,0.775360,0.128859
978863,2021-04-22 02:36:00,0.009262,0.775350,0.128859
978864,2021-04-22 02:37:00,0.009262,0.775375,0.128859
978865,2021-04-22 02:38:00,0.009262,0.775385,0.128861
978866,2021-04-22 02:39:00,0.009263,0.775470,0.128861
978867,2021-04-22 02:40:00,0.009264,0.775490,0.128860
978868,2021-04-22 02:41:00,0.009264,0.775465,0.128860
978869,2021-04-22 02:41:00,NaN,NaN,0.128860


In [166]:
df_forex = pd.read_csv(f"{path_data}df_forex.csv")
df_forex["date"] = [datetime.strptime(date_time, '%Y-%m-%d %H:%M:%S') for date_time in df_forex["date"]]

In [ ]:
country_info = {
    "Australia":{"Currency": "AUD", 
                "Exchange": "SMART",
                "Pairs": aus_adr_underlying_pairs},
    
    "Japan":{"Currency": "JPY", 
             "Exchange": "TSEJ",
             "Pairs": jpn_adr_underlying_pairs},
    
    "HK":{"Currency": "HKD", 
         "Exchange": "SEHK",
         "Pairs": hk_adr_underlying_pairs},
    
}


live_data = {}
def pull_live_data():
    global live_data, country_info, forex_pairs
    
    dict_forex = {}
    for country in forex_pairs:
        currency1, currency2, * = country

        history = "60 S"
        freq = "1 min"
        side = "BID_ASK"

        forex_contract = Contract(symbol = currency1, secType = "CASH", exchange = "IDEALPRO", currency = currency2)
        ib.qualifyContracts(forex_contract)
        df_temp = get_data(forex_contract, history, freq, side)
        
        latest_tick = df_temp.iloc[-1]
        if currency2 == "USD":
            live_data[f"{currency1}.{currency2}"] = (latest_tick["open"] + latest_tick["close"])/2

        else:
            live_data[f"{currency2}.{currency2}"] = 2/(latest_tick["open"] + latest_tick["close"])
            
    for country in country_info.keys():
        currency = country_info[country]["Currency"]
        exchange = country_info[country]["Exchange"]
        pairs_list = country_info[country]["Pairs"]
        
        side = "Trades"

        for ticker_ADR, ticker_underlying in pairs_list:
            contract = Contract(symbol = ticker_ADR, secType = "STK", exchange = "SMART", currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            live_data[ticker_ADR] = latest_tick['close'] # NEED TO MULTIPLY BY ADR RATIO
            
            
            contract = Contract(symbol = ticker_underlying, secType = "STK", exchange = exchange, currency = currency)
            ib.qualifyContracts(contract)
            df_temp = get_data(contract, history, freq, side)
            latest_tick = df_temp.iloc[-1]
            if country == "Australia":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["AUD.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "Japan":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["JPY.USD"] # NEED TO MULTIPLY BY ADR RATIO
            elif country == "HK":
                live_data[ticker_ADR] = latest_tick['close'] * live_data["HKD.USD"] # NEED TO MULTIPLY BY ADR RATIO
        
    
    
def intraday_trader(HK, AUS, JPN, US):
    if HK:
        pairs = country_info["HK"]["Pairs"]
        for ticker_ADR, ticker_underlying in pairs:
            # check Z Score of pair
            
            if (#z score is exit#){
                
                }
    
    

In [ ]:
def z_score

In [ ]:
initialize_forex()

# update date
while True:
    # keep track of what day it is
    today = datetime.date() # ideally the algorithm starts at 8am in the morning
    
    # keep track of how many times we pull data for calibration
    bef_us_pull = False
    bef_us_zscore = False
    bef_asia_pull = False
    bef_jpn_zscore = False
    bef_aus_zscore = False
    bef_hk_zscore = False
    
    # Various Open and Close hours for various markets
    
    US_open = US_opening(today)
    US_close = US_closing(today)
    HK_open = HK_opening(today + timedelta(days = 1))
    HK_close = HK_closing(today + timedelta(days = 1))
    JPN_open = JPN_opening(today + timedelta(days = 1))
    JPN_close = JPN_closing(today + timedelta(days = 1))
    AUS_open = AUS_opening(today + timedelta(days = 1))
    AUS_close = AUS_closing(today + timedelta(days = 1))
    
    # Keep track if pair is open
    
    while now < US_close and now < HK_close and now < JPN_close and now < AUS_close:
        HK_trading = False
        AUS_trading = False
        US_trading = False
        JPN_trading = False
        '''
        US Market
        '''
        # Before US Market Open pull historical data
        if not bef_us_pull and now > US_open - timedelta(hours = 1):
            bef_us_pull = True
            pull_asian_close()
            pull_forex_data()
        
        # Right before US Market Open for Z Score
        if not bef_us_zscore and now > US_open - timedelta(minutes = 4):
            bef_us_zscore = True
            pull_forex_data()
            # Calculate Z score 
            # make trading decision
            
        if US_open < now and now < US_close:
            # Pull minute data to check whether to liquidate positions
            US_trading = True
            pull_live_data()
            intraday_trader(HK_trading, AUS_trading, JPN_trading, US_trading)
            
        if US_close - timedelta(minutes = 10) < now and now < US_close:
            # check pending limit orders and liquidate
                        
        '''
        Asian Markets
        '''
        
        # before asian markets open, pull us market close
        if not bef_asia_pull and now > US_close + timedelta(hours = 1):
            bef_asia_pull = True
            pull_us_close()

        
        # before Hong Kong market open, pull forex to calculate Z score
        if not bef_hk_zscore and now > HK_open - timedelta(minutes = 4):
            bef_hk_zscore = True
            pull_forex_data()
            # calculate Z score and make decision on trade and size
        
        # check on Hong Kong Stocks
        if HK_open < now and now < HK_close:
            HK_trading = True
            
            # Pull minute data to check whether to liquidate positions
            
        if HK_close - timedelta(minutes = 10) < now and now < HK_close:
            # Check pending limit orders and cancel
        
        # before Japan market open, pull forex to calculate Z score
        if not bef_jpn_zscore and now > JPN_opening(today) - timedelta(minutes = 4):
            bef_JPN_zscore = True
            pull_forex_data()
            # calculate Z score
            # make decision on trade and size
        
        # check on Japan stocks
        if JPN_open < now and now < JPN_close:
            # Pull minute data to check whether to liquidate positions
            JPN_trading = True
            
        if JPN_close - timedelta(minutes = 10) < now and now < JPN_close:
            # Check pending limit orders
            
        # before Australia market open, pull forex to calculate Z score            
        if not bef_aus_zscore and now > AUS_opening(today) - timedelta(minutes = 4):
            bef_aus_zscore = True
            pull_forex_data(df_forex, history = "21600 seconds")
        
        # check on Australian Stocks
        if AUS_open < now and now < AUS_close:
            # Pull minute data to check whether to liquidate positions
            AUS_trading = True
            
        if AUS_open - timedelta(minutes = 10) < now and now < AUS_close:
            # Check pending limit orders         

        ib.sleep(120) # Loop every two minutes
        now = datetime.now()

[Contract(secType='STK', conId=13905840, symbol='7751', exchange='TSEJ', primaryExchange='TSEJ', currency='JPY', localSymbol='7751.T', tradingClass='7751')]